In [52]:
import numpy as np
import pandas as pd
from string import ascii_lowercase, ascii_uppercase
import functools
import itertools
import queue
import operator
from copy import deepcopy
from pprint import pprint
import typing

from google.colab import files
uploaded = files.upload()

Saving input.txt to input (1).txt


In [2]:
with open('input.txt') as f:
    x = f.readlines()

In [3]:
move_list = list(filter(lambda y: str(y).startswith("move"), x))
state_list = list(filter(lambda y: operator.not_(str(y).startswith("move")), x))[:-2]

In [4]:
df = pd.DataFrame(move_list, columns = ['raw_moves'])

In [5]:
parsed_moves = df.raw_moves.str.extractall("(\d+)").unstack()
parsed_moves.columns = parsed_moves.columns.droplevel()
parsed_moves = parsed_moves.rename(columns = {0:"count", 1:"to", 2:"from"}).astype('int')

In [6]:
counts, tos, froms = parsed_moves['count'].tolist(), parsed_moves['to'].tolist(), parsed_moves['from'].tolist()
zipped = list(zip(counts, tos, froms))

In [7]:
state_list

['            [C]         [N] [R]    \n',
 '[J] [T]     [H]         [P] [L]    \n',
 '[F] [S] [T] [B]         [M] [D]    \n',
 '[C] [L] [J] [Z] [S]     [L] [B]    \n',
 '[N] [Q] [G] [J] [J]     [F] [F] [R]\n',
 '[D] [V] [B] [L] [B] [Q] [D] [M] [T]\n',
 '[B] [Z] [Z] [T] [V] [S] [V] [S] [D]\n',
 '[W] [P] [P] [D] [G] [P] [B] [P] [V]\n']

In [8]:
with open("new_input.txt", "w+") as f:
    f.write("\n".join(state_list))

In [9]:
df = pd.read_fwf(
    "new_input.txt", 
    header = None,
    colspecs = [(0,3), (4, 7), (8, 11), (12, 15), 
                (16, 19), (20, 23), (24, 27), (28, 31), (32, 35)])

In [10]:
list_of_states = list(df.T.values.tolist())

In [29]:
x = df.T.values
filtered = [list(reversed(list(filter(lambda v: v==v, i)))) for i in x]
filtered

[['[W]', '[B]', '[D]', '[N]', '[C]', '[F]', '[J]'],
 ['[P]', '[Z]', '[V]', '[Q]', '[L]', '[S]', '[T]'],
 ['[P]', '[Z]', '[B]', '[G]', '[J]', '[T]'],
 ['[D]', '[T]', '[L]', '[J]', '[Z]', '[B]', '[H]', '[C]'],
 ['[G]', '[V]', '[B]', '[J]', '[S]'],
 ['[P]', '[S]', '[Q]'],
 ['[B]', '[V]', '[D]', '[F]', '[L]', '[M]', '[P]', '[N]'],
 ['[P]', '[S]', '[M]', '[F]', '[B]', '[D]', '[L]', '[R]'],
 ['[V]', '[D]', '[T]', '[R]']]

In [50]:
# This will be the part a mover, since it can only move one stack at a time. 
def part_a_mover(input_state, zipped_list):
    new_copy = deepcopy(input_state)
    for i, j, k in zipped_list:
        move_blocks = list(reversed(new_copy[j-1][-i:]))
        new_copy[j-1] = new_copy[j-1][:-i]
        new_copy[k-1].extend(move_blocks)
    return new_copy

In [13]:
# So this is actually the part b mover, since it assumes I can move 3 blocks at once, rather than just one at a time.
def part_b_mover(input_state, zipped_list):
    new_copy = deepcopy(input_state)
    for i, j, k in zipped_list:
        move_blocks = new_copy[j-1][-i:]
        new_copy[j-1] = new_copy[j-1][:-i]
        new_copy[k-1].extend(move_blocks)
    return new_copy

In [57]:
def answer_generator(state = typing.List[typing.List]):
    return "".join([i[-1] for i in state]).replace('[', '').replace(']', "")

In [67]:
filtered

[['[W]', '[B]', '[D]', '[N]', '[C]', '[F]', '[J]'],
 ['[P]', '[Z]', '[V]', '[Q]', '[L]', '[S]', '[T]'],
 ['[P]', '[Z]', '[B]', '[G]', '[J]', '[T]'],
 ['[D]', '[T]', '[L]', '[J]', '[Z]', '[B]', '[H]', '[C]'],
 ['[G]', '[V]', '[B]', '[J]', '[S]'],
 ['[P]', '[S]', '[Q]'],
 ['[B]', '[V]', '[D]', '[F]', '[L]', '[M]', '[P]', '[N]'],
 ['[P]', '[S]', '[M]', '[F]', '[B]', '[D]', '[L]', '[R]'],
 ['[V]', '[D]', '[T]', '[R]']]

In [68]:
part_a = part_a_mover(filtered, zipped)
part_b = part_b_mover(filtered, zipped)

In [69]:
answer_generator(part_a)

'LBLVVTVLP'

In [70]:
answer_generator(part_b)

'TPFFBDRJD'